In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models.video as models
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F
import random

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        # self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        # (channel, depth, height, width)
        image = image.transpose(3, 0, 1, 2)
        # if self.transform:
        #     image = self.transform(image)
        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)  
        return image, label

class Modified3DResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.r3d_18()
        self.base_model.fc = nn.Identity()
        num_ftrs = 512
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_ftrs, 2)
        self.features = None
        self.gradients = None
    def forward(self, x):
        x = self.base_model(x)
        self.features = x
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x
    def get_activations(self):
        return self.features
    def get_activations_gradient(self):
        return self.gradients

In [4]:
import torch
import torch.nn as nn

file_path='/Users/dhruv/Desktop/AI Research/best_model (2).pth'

model = Modified3DResNet()
model.load_state_dict(torch.load(file_path,map_location ='cpu'))
model.eval()

Modified3DResNet(
  (base_model): VideoResNet(
    (stem): BasicStem(
      (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (conv2): Sequential(
          (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Sequential(
          (0): Conv3DSimple(64, 64, kernel_s

In [5]:
for name, module in model.named_modules():
    print(name)


base_model
base_model.stem
base_model.stem.0
base_model.stem.1
base_model.stem.2
base_model.layer1
base_model.layer1.0
base_model.layer1.0.conv1
base_model.layer1.0.conv1.0
base_model.layer1.0.conv1.1
base_model.layer1.0.conv1.2
base_model.layer1.0.conv2
base_model.layer1.0.conv2.0
base_model.layer1.0.conv2.1
base_model.layer1.0.relu
base_model.layer1.1
base_model.layer1.1.conv1
base_model.layer1.1.conv1.0
base_model.layer1.1.conv1.1
base_model.layer1.1.conv1.2
base_model.layer1.1.conv2
base_model.layer1.1.conv2.0
base_model.layer1.1.conv2.1
base_model.layer1.1.relu
base_model.layer2
base_model.layer2.0
base_model.layer2.0.conv1
base_model.layer2.0.conv1.0
base_model.layer2.0.conv1.1
base_model.layer2.0.conv1.2
base_model.layer2.0.conv2
base_model.layer2.0.conv2.0
base_model.layer2.0.conv2.1
base_model.layer2.0.relu
base_model.layer2.0.downsample
base_model.layer2.0.downsample.0
base_model.layer2.0.downsample.1
base_model.layer2.1
base_model.layer2.1.conv1
base_model.layer2.1.conv1.0


In [6]:
testlabels=np.load('/Users/dhruv/Downloads/y_holdout.npy')

In [7]:
testdata=np.load('/Users/dhruv/Downloads/x_holdout (2).npy',allow_pickle=True)

In [8]:
testdata.shape

(90, 2)

In [9]:
testdata[45,1].shape

torch.Size([])

In [10]:
testdata[:,0].shape

(90,)

In [11]:
testlabels.shape

(90,)

In [12]:
len(testlabels)

90

In [13]:
#!pip install medcam
from medcam import medcam

In [14]:
model = medcam.inject(model, output_dir="attention_maps", save_maps=True)

In [15]:
test_dataloader = DataLoader(testdata, batch_size=8, shuffle=False)
with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [16]:
#!pip install grad-cam

In [17]:
from pytorch_grad_cam import GradCAM

In [18]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image



In [19]:
x_tensor = torch.tensor(testdata[1,0], dtype=torch.float32)
x_tensor.shape, x_tensor.unsqueeze(0).shape

/var/folders/9v/5cntckpj69vb8ccbz7m453qr0000gp/T/ipykernel_16370/1932081765.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tensor = torch.tensor(testdata[1,0], dtype=torch.float32)


(torch.Size([3, 112, 112, 112]), torch.Size([1, 3, 112, 112, 112]))

In [21]:
# Initialize the model

# Initialize Grad-CAM
grad_cam = GradCAM(model=model, target_layers=model.base_model.layer4[-1].conv2)#layer3[-1].conv2)

# Convert the input tensor to a PyTorch tensor
x_tensor = torch.tensor(testdata[1,0], dtype=torch.float32)

# Generate Grad-CAM heatmap
heatmap = grad_cam(x_tensor.unsqueeze(0))

# Convert the heatmap to a numpy array
#heatmap_np = heatmap.detach().cpu().numpy()

# Visualize the heatmap
# You can overlay the heatmap on the original input video frames for visualization
# For instance, you can use OpenCV or Matplotlib to visualize the heatmap


/var/folders/9v/5cntckpj69vb8ccbz7m453qr0000gp/T/ipykernel_16370/699725035.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tensor = torch.tensor(testdata[1,0], dtype=torch.float32)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
# plt.imshow(testdata[1,0][0,:, :, 78],cmap='gray')

In [ ]:
heatmap.shape

In [ ]:
plt.imshow(heatmap[0][:, :, 0])#, cmap='gray', alpha=0.5)

In [ ]:
targets = [ClassifierOutputTarget(None)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
#grayscale_cam = cam(input_tensor=x_tensor, targets=targets)


# In this example grayscale_cam has only one image in the batch:
grayscale_cam = heatmap[0, :]
visualization = show_cam_on_image(test_nparray, grayscale_cam, use_rgb=True)

# You can also get the model outputs without having to re-inference
model_outputs = cam.outputs

In [ ]:
test_nparray=testdata[1,0].numpy()

In [ ]:
test_nparray.shape

In [ ]:
heatmap_np = heatmap.detach().numpy()


In [ ]:
testdata[1,0].shape